In [580]:
#https://datasource.kapsarc.org/explore/dataset/us-vehicle-fuel-economy-data-1984-2017/table/
import pandas as pd
import re
from sklearn import linear_model
from sklearn.model_selection import KFold
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

np.set_printoptions(threshold=np.inf)


df = pd.read_csv("us-vehicle-fuel-economy-data-1984-2017.csv", sep=';')
pd.set_option('display.max_columns', None)
print(df.shape)
df.head()

(37129, 83)


,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,VClass,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,18,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,Front-Wheel Drive,59007,(FFS),-1,1600,0,Regular,Regular Gasoline,-1,-1,26,0.0,0,0.0,Compact Cars,0.0,0.0,0.0,0,0,10009,0,15,Volkswagen,Jetta III,N,False,0,88,0,0.0,0.0,0.0,0.0,Automatic 4-spd,23.0000,0.0,36.0000,0.0,1993,-1250,NaN,2MODE CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0
1,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,6.0,3.3,Front-Wheel Drive,4410,(FFS),-1,1750,0,Regular,Regular Gasoline,-1,-1,24,0.0,0,0.0,Midsize Cars,0.0,0.0,0.0,0,0,10019,16,16,Buick,Century,Y,False,98,97,0,0.0,0.0,0.0,0.0,Automatic 3-spd,21.1111,0.0,33.0000,0.0,1993,-2000,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0
2,20.600625,0.0,0.0,0.0,14,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,555.437500,16,0.0,0,0.0,0.0,0.0,0.0,8.0,4.0,Rear-Wheel Drive,12071,(GUZZLER) (FFS),-1,2500,0,Premium,Premium Gasoline,-1,-1,20,0.0,0,0.0,Midsize Cars,0.0,0.0,0.0,0,0,10015,0,13,BMW,740i,N,False,0,101,0,0.0,0.0,0.0,0.0,Automatic 5-spd,17.7778,0.0,28.0000,0.0,1993,-5750,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0
3,16.480500,0.0,0.0,0.0,18,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,444.350000,20,0.0,0,0.0,0.0,0.0,0.0,6.0,3.3,Front-Wheel Drive,2611,(FFS),-1,1650,0,Regular,Regular Gasoline,-1,-1,24,0.0,0,0.0,Midsize Cars,0.0,0.0,0.0,0,0,10042,0,16,Dodge,Dynasty,N,False,0,99,0,0.0,0.0,0.0,0.0,Automatic 4-spd,22.0000,0.0,33.3333,0.0,1993,-1500,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0
4,16.480500,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,444.350000,20,0.0,0,0.0,0.0,0.0,0.0,6.0,3.8,Front-Wheel Drive,4400,(FFS),-1,1650,0,Regular,Regular Gasoline,-1,-1,26,0.0,0,0.0,Midsize Cars,0.0,0.0,0.0,0,0,10023,16,16,Buick,Regal,Y,False,95,100,0,0.0,0.0,0.0,0.0,Automatic 4-spd,21.0000,0.0,36.0000,0.0,1993,-1500,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0


In [581]:
#resort the columns by identifying information at the front, then alphabetically
head = ['id', 'make', 'model', 'year', 'trany']
cols = [x for x in sorted((df)) if x not in head]
cols = head + cols
df = df[cols].sort_values(by='id')

#view the newly organized dataset
df.head()

,id,make,model,year,trany,UCity,UCityA,UHighway,UHighwayA,VClass,atvType,barrels08,barrelsA08,c240Dscr,c240bDscr,charge120,charge240,charge240b,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,createdOn,cylinders,displ,drive,engId,eng_dscr,evMotor,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,fuelType2,ghgScore,ghgScoreA,guzzler,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,lv2,lv4,mfrCode,modifiedOn,mpgData,phevBlended,phevCity,phevComb,phevHwy,pv2,pv4,range,rangeA,rangeCity,rangeCityA,rangeHwy,rangeHwyA,sCharger,startStop,tCharger,trans_dscr,youSaveSpend
33941,1,Alfa Romeo,Spider Veloce 2000,1985,Manual 5-spd,23.3333,0.0,35.000,0.0,Two Seaters,NaN,15.695714,0.0,NaN,NaN,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,2013-01-01,4.0,2.0,Rear-Wheel Drive,9011,(FFS),NaN,-1,1600,0,Regular,Regular Gasoline,NaN,-1,-1,NaN,25,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN,2013-01-01,Y,False,0,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,-1250
2979,2,Bertone,X1/9,1985,Manual 5-spd,25.0000,0.0,36.000,0.0,Two Seaters,NaN,14.982273,0.0,NaN,NaN,0.0,0.0,0.0,20,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,403.954545,22,0.0,0,0.0,0.0,0.0,0.0,2013-01-01,4.0,1.5,Rear-Wheel Drive,12710,NaN,NaN,-1,1500,0,Regular,Regular Gasoline,NaN,-1,-1,NaN,26,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN,2013-01-01,Y,False,0,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,-750
27386,3,Chevrolet,Corvette,1985,Automatic 4-spd,18.0000,0.0,29.000,0.0,Two Seaters,NaN,19.388824,0.0,NaN,NaN,0.0,0.0,0.0,15,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,522.764706,17,0.0,0,0.0,0.0,0.0,0.0,2013-01-01,8.0,5.7,Rear-Wheel Drive,4185,(350 V8) (FFS),NaN,-1,1950,0,Regular,Regular Gasoline,NaN,-1,-1,NaN,21,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN,2013-01-01,Y,False,0,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,-3000
6311,4,Chevrolet,Corvette,1985,Manual 4-spd,18.0000,0.0,28.000,0.0,Two Seaters,NaN,19.388824,0.0,NaN,NaN,0.0,0.0,0.0,15,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,522.764706,17,0.0,0,0.0,0.0,0.0,0.0,2013-01-01,8.0,5.7,Rear-Wheel Drive,4185,(350 V8) (FFS),NaN,-1,1950,0,Regular,Regular Gasoline,NaN,-1,-1,NaN,20,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN,2013-01-01,N,False,0,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,-3000
14021,5,Nissan,300ZX,1985,Automatic 4-spd,18.0000,0.0,24.359,0.0,Two Seaters,NaN,20.600625,0.0,NaN,NaN,0.0,0.0,0.0,15,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,555.437500,16,0.0,0,0.0,0.0,0.0,0.0,2013-01-01,6.0,3.0,Rear-Wheel Drive,38043,"(GUZZLER) (FFS,TRBO)",NaN,-1,2100,0,Regular,Regular Gasoline,NaN,-1,-1,T,18,0.0,0,0.0,0.0,0.0,0.0,23,50,0,0,NaN,2013-01-01,N,False,0,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,T,2MODE,-3750


In [582]:
#Redundant or non-useful attributes we'll remove are listed here.
#We remove fuelCost attributes on the basis that fuel prices change over time,
#so these attributes only encapsulate the fuelCost at a given time.
#We also remove all EPA emissions scores as they are a direct function of a car's
#fuelType and co2 tailpipe output.
drop = ['UCity', 'UCityA', 'atvType', 'barrels08', 'barrelsA08', 'fuelType', \
       'UHighway', 'UHighwayA', 'createdOn', 'modifiedOn', 'cityA08U', 'city08U', \
       'co2', 'co2A', 'comb08U', 'combA08U', 'engId', 'eng_dscr', 'fuelCost08', \
       'fuelCostA08', 'guzzler', 'highway08U', 'highwayA08U', 'mfrCode', \
       'mpgData', 'trans_dscr', 'VClass', 'youSaveSpend', 'hlv', 'hpv', 'lv2', \
       'lv4', 'pv2', 'pv4', 'ghgScore', 'ghgScoreA', 'feScore']

#Collect columns which are are mostly empty.
#The print statement can be uncommented to see which
#rows have empty values and by what percentage.
empty_attrs = []
empty_vals = [np.NaN, 0, -1]
for col_name in cols:
    col = df[col_name]
    non_empty = [x for x in col if x not in empty_vals]
    #print(col_name, len(non_empty) / len(col))
    if len(non_empty) / len(col) < 0.01:
        empty_attrs.append(col_name)
        
#remove unwanted attributes
drop = list(set().union(empty_attrs, drop))
df = df.drop(drop, axis = 1)
df.head()

,id,make,model,year,trany,city08,cityA08,co2TailpipeAGpm,co2TailpipeGpm,comb08,combA08,cylinders,displ,drive,evMotor,fuelType1,fuelType2,highway08,highwayA08,rangeA,sCharger,startStop,tCharger
33941,1,Alfa Romeo,Spider Veloce 2000,1985,Manual 5-spd,19,0,0.0,423.190476,21,0,4.0,2.0,Rear-Wheel Drive,NaN,Regular Gasoline,NaN,25,0,NaN,NaN,NaN,NaN
2979,2,Bertone,X1/9,1985,Manual 5-spd,20,0,0.0,403.954545,22,0,4.0,1.5,Rear-Wheel Drive,NaN,Regular Gasoline,NaN,26,0,NaN,NaN,NaN,NaN
27386,3,Chevrolet,Corvette,1985,Automatic 4-spd,15,0,0.0,522.764706,17,0,8.0,5.7,Rear-Wheel Drive,NaN,Regular Gasoline,NaN,21,0,NaN,NaN,NaN,NaN
6311,4,Chevrolet,Corvette,1985,Manual 4-spd,15,0,0.0,522.764706,17,0,8.0,5.7,Rear-Wheel Drive,NaN,Regular Gasoline,NaN,20,0,NaN,NaN,NaN,NaN
14021,5,Nissan,300ZX,1985,Automatic 4-spd,15,0,0.0,555.437500,16,0,6.0,3.0,Rear-Wheel Drive,NaN,Regular Gasoline,NaN,18,0,NaN,NaN,NaN,T


In [583]:
#This commented out 3 lines below can be used to print out the range of values for the 
#categorical variables we wanted to create dummies for and was used to help
#write the dummy creator code below it.

#cats = ['trany', 'drive', 'sCharger', 'tCharger', 'startStop', 'fuelType1', 'fuelType2']
#for cat in cats:
#    print(cat, df[cat].unique())

#create our dummy vars for each row of data and impute some values
for ix, row in df.iterrows():
    trans = row['trany']
    drive = row['drive']
    f1 = row['fuelType1']
    f2 = row['fuelType2']
    sCharge = row['sCharger']
    tCharge = row['tCharger']
    ss = row['startStop']
    evM = row['evMotor']
    cyl = row['cylinders']
    displ = row['displ']
     
    #transmission will be split into 3 attributes: the dummies manual? and automatic?, 
    #and the numeric attribute gears. For transmissions with no recorded gears, we
    #sample from the data for that year to impute a value.
    if not pd.isnull(trans):
        df.set_value(ix, 'manual?', 1 if 'Manual' in trans else 0)
        df.set_value(ix, 'automatic?', 1 if 'Auto' in trans else 0)
        gears = re.findall(r'\d+', trans)
        if len(gears) > 0:
            df.set_value(ix, 'gears', gears[0])
        else:
            gears = np.NaN
            while pd.isnull(gears):
                gears = list(df.loc[(df['year'] == row['year'])]['gears'].sample(n=1))[0]
            df.set_value(ix, 'gears', gears)
    #A handful of vehicles have no recorded transmission. We are not sure why, but we treat 
    #these records as missing at random and impute them by sampling from the distribution of
    #values from the year of data in question.
    else:
        auto = np.NaN
        while pd.isnull(auto):
            auto = list(df.loc[(df['year'] == row['year'])]['automatic?'].sample(n=1))[0]
        df.set_value(ix, 'manual?', 1 if auto == 1 else 0)
        df.set_value(ix, 'automatic?', 0 if auto == 1 else 1)
        gears = np.NaN
        while pd.isnull(gears):
            gears = list(df.loc[(df['year'] == row['year'])]['gears'].sample(n=1))[0]
        df.set_value(ix, 'gears', gears)
    
    
    #next we'll tidy up drive. drive is recorded inconsistently, sometimes as front wheel,
    #rear wheel, or all wheel, or sometimes simply as 2 wheel or 4 wheel. We simplify
    #the attribute to the simple dummies 2-wheel?, 4-wheel?, rear? and front?
    if not pd.isnull(drive):
        df.set_value(ix, '2WheelDrive?', 1 if 'Rear' in drive or 'Front' in drive or \
                     '2' in drive else 0)
        df.set_value(ix, '4WheelDrive?', 1 if 'All' in drive or '4' in drive else 0)
        df.set_value(ix, 'rear?', 1 if 'Rear' in drive else 0)
        df.set_value(ix, 'front?', 1 if 'Front' in drive else 0)
    #As was the case for transmission, some vehicles are missing a drive variables. This accounts
    #for nearly 4% of the data. Instead of trying to impute values for so much data for which
    #we cannot discern the reason for missingness, we set their dummy attributes to 0.
    else:
        df.set_value(ix, '2WheelDrive?', 0)
        df.set_value(ix, '4WheelDrive?', 0)
        df.set_value(ix, 'rear?', 0)
        df.set_value(ix, 'front?', 0)
        
        
    #next are the the fuelType vars. We'll split fuelType1 into dummies based on type
    #and create a single fuelType2? dummy from fuelType2.
    df.set_value(ix, 'regularGas?', 1 if 'Regular' in f1 else 0)
    df.set_value(ix, 'premiumGas?', 1 if 'Premium' in f1 else 0)
    df.set_value(ix, 'midgradeGas?', 1 if 'Midgrade' in f1 else 0)
    df.set_value(ix, 'diesel?', 1 if 'Diesel' in f1 else 0)
    df.set_value(ix, 'naturalGas?', 1 if 'Natural' in f1 else 0)
    df.set_value(ix, 'electric?', 1 if 'Electric' in f1 else 0)
    df.set_value(ix, 'fuelType2?', 1 if type(f2) == str else 0)
    
    #next dummies for the turbo and super charged and startStop attributes
    df.set_value(ix, 'tCharger', 1 if type(tCharge) == str else 0)
    df.set_value(ix, 'sCharger', 1 if type(sCharge) == str else 0)
    df.set_value(ix, 'startStop', 1 if type(ss) == str and ss == 'Y' else 0)
    
    
    #That's it for the dummy variables. Now we'll clean evMotor which has NaN's
    #and non standard syntax instead of a simple int value
    if not pd.isnull(evM):
        kw_hrs = re.findall(r'\d+', evM)
        df.set_value(ix, 'kwHrs', kw_hrs[0])
    else:
        df.set_value(ix, 'kwHrs', 0)
    
    #Finally, we impute displ and cylinders for a handful of cars missing this data. We recognize 
    #electric cars have no cylinders or displ, so we impute 0 for electric cars missing this data.
    #We treat other missing records as missing at random and sample from the dataset to impute
    #these values.
    if pd.isnull(displ):
        if 'Electric' in f1:
            df.set_value(ix, 'displ', 0)
        else:
            while pd.isnull(displ):
                displ = list(df.loc[(df['year'] == row['year'])]['displ'].sample(n=1))[0]
            df.set_value(ix, 'displ', displ)
    if pd.isnull(cyl):
        if 'Electric' in f1:
            df.set_value(ix, 'cylinders', 0)
        else:
            while pd.isnull(cyl):
                cyl = list(df.loc[(df['year'] == row['year'])]['cylinders'].sample(n=1))[0]
            df.set_value(ix, 'cylinders', cyl)
    
    
#Finally delete our categorical attributes we no longer need and sort the attributes as before
drop = ['fuelType1', 'drive', 'trany', 'fuelType2', 'evMotor']
df = df.drop(drop, axis = 1)
head = ['id', 'make', 'model', 'year']
cols = [x for x in sorted((df)) if x not in head]
cols = head + cols
df = df[cols].sort_values(by='id')
df.head()

,id,make,model,year,2WheelDrive?,4WheelDrive?,automatic?,city08,cityA08,co2TailpipeAGpm,co2TailpipeGpm,comb08,combA08,cylinders,diesel?,displ,electric?,front?,fuelType2?,gears,highway08,highwayA08,kwHrs,manual?,midgradeGas?,naturalGas?,premiumGas?,rangeA,rear?,regularGas?,sCharger,startStop,tCharger
33941,1,Alfa Romeo,Spider Veloce 2000,1985,1.0,0.0,0.0,19,0,0.0,423.190476,21,0,4.0,0.0,2.0,0.0,0.0,0.0,5,25,0,0.0,1.0,0.0,0.0,0.0,NaN,1.0,1.0,0,0,0
2979,2,Bertone,X1/9,1985,1.0,0.0,0.0,20,0,0.0,403.954545,22,0,4.0,0.0,1.5,0.0,0.0,0.0,5,26,0,0.0,1.0,0.0,0.0,0.0,NaN,1.0,1.0,0,0,0
27386,3,Chevrolet,Corvette,1985,1.0,0.0,1.0,15,0,0.0,522.764706,17,0,8.0,0.0,5.7,0.0,0.0,0.0,4,21,0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,0,0,0
6311,4,Chevrolet,Corvette,1985,1.0,0.0,0.0,15,0,0.0,522.764706,17,0,8.0,0.0,5.7,0.0,0.0,0.0,4,20,0,0.0,1.0,0.0,0.0,0.0,NaN,1.0,1.0,0,0,0
14021,5,Nissan,300ZX,1985,1.0,0.0,1.0,15,0,0.0,555.437500,16,0,6.0,0.0,3.0,0.0,0.0,0.0,4,18,0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,0,0,1


In [584]:
#Creating a linear regression model to predict city08
#remove unrelated predictors
predictors = df.drop(['id', 'make', 'model', 'city08', 'cityA08', 'co2TailpipeAGpm', 'comb08', 'combA08', 'highwayA08', 'rangeA'], axis = 1).astype(float)
predictors.head()


,year,2WheelDrive?,4WheelDrive?,automatic?,co2TailpipeGpm,cylinders,diesel?,displ,electric?,front?,fuelType2?,gears,highway08,kwHrs,manual?,midgradeGas?,naturalGas?,premiumGas?,rear?,regularGas?,sCharger,startStop,tCharger
33941,1985.0,1.0,0.0,0.0,423.190476,4.0,0.0,2.0,0.0,0.0,0.0,5.0,25.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2979,1985.0,1.0,0.0,0.0,403.954545,4.0,0.0,1.5,0.0,0.0,0.0,5.0,26.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
27386,1985.0,1.0,0.0,1.0,522.764706,8.0,0.0,5.7,0.0,0.0,0.0,4.0,21.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
6311,1985.0,1.0,0.0,0.0,522.764706,8.0,0.0,5.7,0.0,0.0,0.0,4.0,20.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
14021,1985.0,1.0,0.0,1.0,555.437500,6.0,0.0,3.0,0.0,0.0,0.0,4.0,18.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


In [585]:
#add a constant term to our regression
x = sm.add_constant(predictors.values)

#convert to np.array
y = df['city08'].as_matrix()

#use ordinary least squares, fit model and print results of regression
model = sm.OLS(y, x)
result = model.fit()
print(result.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.936
Model:                            OLS   Adj. R-squared:                  0.936
Method:                 Least Squares   F-statistic:                 2.588e+04
Date:                Wed, 17 May 2017   Prob (F-statistic):               0.00
Time:                        02:57:51   Log-Likelihood:                -71160.
No. Observations:               37129   AIC:                         1.424e+05
Df Residuals:                   37107   BIC:                         1.426e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          6.6015      1.660      3.977      0.0

In [586]:
#after getting our first model, we want to see if it can be improved
#first let's see if our model has high multicollinearity
vif = [variance_inflation_factor(x, ix) for ix in range(x.shape[1])]
print (vif)

C:\Users\Erica\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1181: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\Erica\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


[0.0, 2.8377481615781552, 22.358239106010629, 8.172653684489033, inf, 9.7115782068948775, 6.6954666315262372, inf, 8.1710154371926862, inf, 19.245946246186097, 1.1019124425625757, 3.0375146336947227, 9.4672248391826965, 1.427515818924662, inf, inf, inf, inf, 17.932758781185061, inf, 1.0849568609368005, 1.5631590537161508, 1.4189295035119454]


In [587]:
#from the output, we can see that there are many values of vif > 5 and even > 10
#so multicollinearity is definitely an issue

#so what we will do is drop the variable with the highest multicollinearity
#and then recalculate the vifs and repeat these same steps until we no longer have
#any vif > 5 in our model
def calculate_vif_(X, threshold):
    dropped=True
    while dropped:
        dropped=False
        vif = [variance_inflation_factor(X.values, ix) for ix in range(X.shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > threshold:
            print('dropping \'' + X.columns[maxloc] + '\' at index: ' + str(maxloc))
            X = X.drop(X.columns[maxloc], axis = 1)
            dropped=True

    return X

predictors = calculate_vif_(predictors, 5.0)

C:\Users\Erica\Anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:167: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


dropping 'automatic?' at index: 3
dropping 'regularGas?' at index: 18
dropping 'year' at index: 0
dropping 'cylinders' at index: 3
dropping '2WheelDrive?' at index: 0
dropping 'co2TailpipeGpm' at index: 1
dropping 'gears' at index: 6
dropping 'highway08' at index: 6
dropping 'displ' at index: 2


In [588]:
#this is our new dataframe of predictors (to predict city08)
predictors.head()

,4WheelDrive?,diesel?,electric?,front?,fuelType2?,kwHrs,manual?,midgradeGas?,naturalGas?,premiumGas?,rear?,sCharger,startStop,tCharger
33941,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2979,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
27386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6311,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
14021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [589]:
#after getting rid of all of the highly correlated variables these are the new scores, which are much better than before
vif = [variance_inflation_factor(predictors.values, ix) for ix in range(predictors.values.shape[1])]
print (vif)

#now after getting rid our multicollinearity issue, we rerun our regression
x = sm.add_constant(predictors.values)
model = sm.OLS(y, x)
result = model.fit()
print(result.summary())



[1.3249273649861137, 1.069462084042869, 1.0762943243024818, 1.3312508666965808, 1.0707115765523332, 1.4199502175733587, 1.4882732905257425, 1.0051549286361854, 1.004085065026598, 1.845699416043076, 1.4905568580990596, 1.0956266932649545, 1.4854835516090632, 1.4793066445163716]
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     5089.
Date:                Wed, 17 May 2017   Prob (F-statistic):               0.00
Time:                        02:58:09   Log-Likelihood:            -1.0233e+05
No. Observations:               37129   AIC:                         2.047e+05
Df Residuals:                   37114   BIC:                         2.048e+05
Df Model:                          14                                         
Covariance 

In [593]:
#after looking at our results summary, we need to remove one more predictor, x8,
#whose p value > .05, which means it is not significant
predictors = predictors.drop(predictors.columns[[7]], axis = 1)

#now let's rerun the regression one more time
x = sm.add_constant(predictors.values)
model = sm.OLS(y, x)
result = model.fit()
print(result.summary())

print(predictors.head(5))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     6457.
Date:                Wed, 17 May 2017   Prob (F-statistic):               0.00
Time:                        03:13:42   Log-Likelihood:            -1.0237e+05
No. Observations:               37129   AIC:                         2.048e+05
Df Residuals:                   37117   BIC:                         2.049e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         16.6584      0.096    173.715      0.0

In [591]:
#cross-validation 

X = predictors.values
y = df['city08'].as_matrix()

kf = KFold(n_splits=10)
#print(kf)  

regr = linear_model.LinearRegression()
model = regr.fit(X,y)
print(model.coef_)
print(model.intercept_)
print(model.residues_)

predictions = regr.predict(X)

# print(predictions)
 

# for train_index, test_index in kf.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

KFold(n_splits=10, random_state=None, shuffle=False)
[ -1.81108580e+00   4.31576715e+00   7.46232114e+01   3.52665942e+00
  -4.10030637e-01   3.00406248e-02   1.67756645e+00  -1.01682884e+00
  -4.44064456e-01  -1.97116979e+00  -1.04441941e+00   4.84871280e+00
   6.31480308e-01]
16.6693856021
538358.019102


C:\Users\Erica\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function residues_ is deprecated; ``residues_`` is deprecated and will be removed in 0.19
  warnings.warn(msg, category=DeprecationWarning)
